<a href="https://colab.research.google.com/github/AnoVando/Work2/blob/master/supplier_tdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sparse_dot_topn

In [0]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Import your data to a Pandas.DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/AnoVando/Work2/master/supplier_data_new.csv', encoding = 'ISO-8859-1')

In [0]:
df.drop(df[df.PACCAR_ID == 'UNK'].index, inplace=True)

In [0]:
df = df.applymap(str)

In [0]:
df['Address'] = df['Address'].str.upper()
df['Name'] = df['Name'].str.upper()
df['City'] = df['City'].str.upper()
df['Region'] = df['Region'].str.upper()
df['Country'] = df['Country'].str.upper()

In [354]:
df.head()

,System,System_ID,PACCAR_ID,Address,Name,City,Region,Country,Postal_Code
0,SAP S4,SAP S4,11526AA,21 TIDEMAN DRIVE,ROCHLING ENGINEERING PLAST,ORANGEVILLE,ON,CA,L9W 3K3
1,KW Australia,nan,6250675,"2865 LIUXIANG ROAD, JIADING DISTRICT",IBCC PROTECH INC,SHANGHAI,NAN,CN,201818
2,SAP S4,SAP S4,24022LA,AV. DE LA MONTAÑA #128,"EATON TRADING CO., S. DE",QUERETARO,QRO,MX,76220
3,SAP S4,SAP S4,21722DC,SUITE 110 11101 METRO AIRPORT CENT,POLYBOND- EXPEDITORS,DETROIT,MI,US,48174
4,KW Australia,nan,6251295,#05-56 VERTEX,LITTELFUSE FAR EAST,NAN,NAN,SG,nan


In [0]:
df['Full_Address'] = df['Address'].map(str) + df['City'].map(str) + df['Region'].map(str)

In [0]:
# Grab the column you'd like to group, filter out duplicate values
# and make sure the values are Unicode
vals = df['Full_Address'].unique().astype('U')


# Write a function for cleaning strings and returning an array of ngrams
def ngrams_analyzer(string):
    string = re.sub(r'[,-./]', r'', string)
    ngrams = zip(*[string[i:] for i in range(3)])  # N-Gram length
    return [''.join(ngram) for ngram in ngrams]

# Construct your vectorizer for building the TF-IDF matrix
vectorizer = TfidfVectorizer(analyzer=ngrams_analyzer)

# Build the matrix!!!
tf_idf_matrix = vectorizer.fit_transform(vals)

In [0]:
# Import IGN's awesome_cossim_topn module
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import rand
from sparse_dot_topn import awesome_cossim_topn


# The arguments for awesome_cossim_topn are as follows:
### 1. Our TF-IDF matrix
### 2. Our TF-IDF matrix transposed (allowing us to build a pairwise cosine matrix)
### 3. A top_n filter, which allows us to filter the number of matches returned, which isn't useful for our purposes
### 4. This is our similarity threshold. Only values over 0.8 will be returned
cosine_matrix = awesome_cossim_topn(
  tf_idf_matrix,
  tf_idf_matrix.transpose(),
  vals.size,
  0.9)


In [0]:
# Build a coordinate matrix from a cosine matrix
coo_matrix = cosine_matrix.tocoo()

# Instaniate our lookup hash table
group_lookup = {}


def find_group(row, col):
    # If either the row or the col string have already been given
    # a group, return that group. Otherwise return none
    if row in group_lookup:
        return group_lookup[row]
    elif col in group_lookup:
        return group_lookup[col]
    else:
        return None


def add_vals_to_lookup(group, row, col):
    # Once we know the group name, set it as the value
    # for both strings in the group_lookup
    group_lookup[row] = group
    group_lookup[col] = group


def add_pair_to_lookup(row, col):
    # in this function we'll add both the row and the col to the lookup
    group = find_group(row, col)  # first, see if one has already been added
    if group is not None:
        # if we already know the group, make sure both row and col are in lookup
        add_vals_to_lookup(group, row, col)
    else:
        # if we get here, we need to add a new group.
        # The name is arbitrary, so just make it the row
        add_vals_to_lookup(row, row, col)

# for each row and column in coo_matrix
# if they're not the same string add them to the group lookup
for row, col in zip(coo_matrix.row, coo_matrix.col):
    if row != col:
        # Note that what is passed to add_pair_to_lookup is the string at each index
        # (eg: the names in the legal_name column) not the indices themselves
        add_pair_to_lookup(vals[row], vals[col])

In [0]:
df['Group'] = df['Full_Address'].map(group_lookup).fillna(df['Full_Address'])

#df.to_csv('./dol-data-grouped.csv')

In [0]:
df['Occurences'] = df.groupby('Group')['Group'].transform('count')

In [0]:
df.drop(columns='Full_Address', inplace=True)

In [362]:
df.head()

,System,System_ID,PACCAR_ID,Address,Name,City,Region,Country,Postal_Code,Group,Occurences
0,SAP S4,SAP S4,11526AA,21 TIDEMAN DRIVE,ROCHLING ENGINEERING PLAST,ORANGEVILLE,ON,CA,L9W 3K3,21 TIDEMAN DRIVEORANGEVILLEON,6
1,KW Australia,nan,6250675,"2865 LIUXIANG ROAD, JIADING DISTRICT",IBCC PROTECH INC,SHANGHAI,NAN,CN,201818,"2865 LIUXIANG ROAD, JIADING DISTRICTSHANGHAINAN",1
2,SAP S4,SAP S4,24022LA,AV. DE LA MONTAÑA #128,"EATON TRADING CO., S. DE",QUERETARO,QRO,MX,76220,AV. DE LA MONTAÑA #128QUERETAROQRO,2
3,SAP S4,SAP S4,21722DC,SUITE 110 11101 METRO AIRPORT CENT,POLYBOND- EXPEDITORS,DETROIT,MI,US,48174,SUITE 110 11101 METRO AIRPORT CENTDETROITMI,1
4,KW Australia,nan,6251295,#05-56 VERTEX,LITTELFUSE FAR EAST,NAN,NAN,SG,nan,#05-56 VERTEXNANNAN,1


In [0]:
from google.colab import files

df.to_excel('supplier_data_address_90.xlsx')